# Notebook da Aula 14

In [ ]:
import networkx as nx
import freeman as fm

## Parâmetros para variar

In [ ]:
POPULATION = 10
FRIENDSHIP = 0.5

PRESSURE_1 = 1 # quando X amigo de Y, X amigo de Z, Y inimigo de Z, essa é a pressão sobre YZ
PRESSURE_2 = 1 # quando X amigo de Y, X inimigo de Z, Y amigo de Z, essa é a pressão sobre YZ
PRESSURE_3 = 1 # quando todos são inimigos mútuos, essa é a pressão sobre todas as três arestas

## Algumas funções de conveniência

In [ ]:
from random import random


def colorize_edges(g):
    for n, m in g.edges:
        if g.edges[n, m]['friendship']:
            g.edges[n, m]['color'] = (0, 0, 255)
        else:
            g.edges[n, m]['color'] = (255, 0, 0)


def generate_graph():
    g = fm.Graph(nx.complete_graph(POPULATION))

    for n, m in g.edges:
        if random() < FRIENDSHIP:
            g.edges[n, m]['friendship'] = True
        else:
            g.edges[n, m]['friendship'] = False

    g.set_all_nodes('size', 10)
    colorize_edges(g)
    return g


g = generate_graph()
g.draw()

## Simulação

In [ ]:
from random import choice


with fm.Animation() as a:
    g = generate_graph()

    while True:
        for n, m in g.edges:
            g.edges[n, m]['pressure'] = 0

        for n, m, l in g.triads():
            friendships = 0
            if g.edges[n, m]['friendship']:
                friendships += 1
            if g.edges[n, l]['friendship']:
                friendships += 1
            if g.edges[m, l]['friendship']:
                friendships += 1

            if friendships % 2 == 0:
                if friendships == 0:
                    g.edges[n, m]['pressure'] += PRESSURE_3
                    g.edges[n, l]['pressure'] += PRESSURE_3
                    g.edges[m, l]['pressure'] += PRESSURE_3
                else:
                    if g.edges[n, m]['friendship']:
                        g.edges[n, m]['pressure'] += PRESSURE_2
                        if g.edges[n, l]['friendship']:
                            g.edges[n, l]['pressure'] += PRESSURE_2
                            g.edges[m, l]['pressure'] += PRESSURE_1
                        else:
                            g.edges[n, l]['pressure'] += PRESSURE_2
                            g.edges[m, l]['pressure'] += PRESSURE_2
                    else:
                        g.edges[n, m]['pressure'] += PRESSURE_1
                        g.edges[n, l]['pressure'] += PRESSURE_2
                        g.edges[m, l]['pressure'] += PRESSURE_2

        pressured = [(n, m) for n, m in g.edges if g.edges[n, m]['pressure'] > 0]

        if pressured:
            n, m = max(pressured, key=lambda e: g.edges[e[0], e[1]]['pressure'])
            g.edges[n, m]['friendship'] = not g.edges[n, m]['friendship']
            colorize_edges(g)

            g.set_each_edge('weight', lambda n, m: int(g.edges[n, m]['friendship']))
            g.move('step', iterations=10)
            a.rec(g)
        else:
            break